# Load and BIO label

This script loads the data and its labels from given paths/places (see configs.ini file), joins the data with BIO labeling information, and saves the resulting file(s) to respective subfolder in the folder *_data*. You do not need a gpu to do preprocessing of data.

In [1]:
# install missing libraries if any
import sys
!{sys.executable} -m pip install dask[dataframe] pandas pyarrow seaborn spacy nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 937.8/937.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
print("hello worls!")

hello worls!


In [3]:
# import libraries
import ast # for converting list as a string to a list
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
import configparser
config = configparser.ConfigParser() #init
config.read('../configs.ini') # init config with values from configs.ini
import hashlib
import nltk
from nltk.tokenize import word_tokenize
# nltk.download('punkt')
import numpy as np
import os
print(os.getcwd()) # show current "local" folder location
import pandas as pd
import re
import spacy
nlp = spacy.blank('fi') # init spacy with Finnish
import string


/mnt/batch/tasks/shared/LS_root/mounts/clusters/ext131165225/code/Users/EXT13116522/base_ner_refactored/scripts_to_preprocess_data


## Parameters

In [4]:
# VARIABLES
# experiment of interest (experiment name must match the section name in configs.ini)
#experiment = 'loneliness'
#experiment = 'incontinence_v5'
#experiment = 'feeding_20240515'
#experiment = 'mobility_v5'
#experiment = 'mobility_v5.C.original'
#experiment = 'Falling_NER_v3_20231114'# DONE
#experiment = 'Mobility_2404_20240619'
#experiment = 'Loneliness_beta0_20231123'
experiment = 'Incontinence_NER_v5_20231208'

# CONSTANTS from config
# to save data
data_folder = config[experiment]['data_folder'] # must exist
data_subfolder = config[experiment]['data_subfolder'] # must exist
journals_subfolder = config[experiment]['journals_subfolder'] # the key should exist

# path to save folder
data_save_folder_path = os.path.join(data_folder, data_subfolder)

In [5]:
print('data_save_folder_path', data_save_folder_path)
print('journals_subfolder', journals_subfolder)
print('type', type(journals_subfolder))
print('length', len(journals_subfolder))

data_save_folder_path ../_data/data_Incontinence_NER_v5_20231208
journals_subfolder incontinence_journals
type <class 'str'>
length 21


## Main code

### Load labels and journals data

In [6]:
# You might encounter UserErrorException
#"""UserErrorException: UserErrorException:Message: You are currently logged-in to 
# <tenant_id> tenant. You don't have access to <supscription_id> subscription, please check 
# if it is in this tenant. All the subscriptions that you have access to in this tenant are..."""
# 
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

workspace = Workspace(
    config['default']['SUBSCRIPTION_ID'], 
    config['default']['RESOURCE_GROUP'], 
    config['default']['WORKSPACE_NAME'])

In [7]:
# load dataset
dataset = Dataset.get_by_name(workspace, name=config[experiment]['data_name'])
# save dataset to a TEMPORARY folder (if folder already exists and overwrite is False then throws error)
temporary_folder_path = data_save_folder_path + '_tmp'
try:
    dataset.download(target_path=temporary_folder_path, overwrite=False)
#except UserErrorException as e: #UserErrorException is not in this environment? 
except Exception as e:
    print(f"Please, remove the temporary folder from the folder structure manually. It is called {temporary_folder_path}.")
    print("#------------------#")
    print(f"Encountered an exception. Error was: {e}")

{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}


In [8]:
# check contents of the tmp folder
hasFolder=False
dataset_directory_name = None
for root, dirs, files in os.walk(temporary_folder_path, topdown=False):
    if len(dirs) == 1: # is there a subfolder?
        print("# we got a folder!")
        dataset_directory_name = dirs[0]
        hasFolder = True
    else: pass

# Did we fail to get the subfolder name?
if hasFolder is False and dataset_directory_name is None:
    print("There was no folder!")
    # we get the folder name from configuration
    dataset_directory_name = journals_subfolder
else: pass

print('dataset_directory_name', dataset_directory_name)

There was no folder!
dataset_directory_name incontinence_journals


In [9]:
def delete_non_empty_folder(top):
    """Remove folder based on the path"""
    for root, dirs, files in os.walk(top, topdown=False):
        for name in files: # delete files
            os.remove(os.path.join(root, name))
        for name in dirs: # delete directories
            os.rmdir(os.path.join(root, name))
    if os.path.exists(top):
        # delete the empty tmp folder
        os.removedirs(top)

In [10]:
# if there is a subfolder with some name
if hasFolder and dataset_directory_name is not None:
    dataset.download(target_path=data_save_folder_path, overwrite=True)
    print(f"downloaded to: {data_save_folder_path}")
    # delete tmp folder    
    delete_non_empty_folder(os.path.join(temporary_folder_path, dataset_directory_name)) # delete files from folder
    print(f"deleted tmp folder: {os.path.join(temporary_folder_path, dataset_directory_name)}")
    delete_non_empty_folder(temporary_folder_path) # delete any files outside the subfolder (tmp folder)
    print(f"deleted tmp folder: {os.path.join(temporary_folder_path)}")
else: 
    print(f"HasFolder was {hasFolder} and dataset_directory_name was {dataset_directory_name}")

HasFolder was False and dataset_directory_name was incontinence_journals


In [11]:
if hasFolder is False and dataset_directory_name is not None:
    # we got files without a folder, thus all journal 
    # files must be moved to specified subfolder.
    for root, dirs, files in os.walk(temporary_folder_path, topdown=False):
        # fetch the files again
        if len(dirs) == 0: # there should be no folders, instead many files
            print(f"we got files without a folder in {temporary_folder_path}")
            # select journal related files by excluding possible label file.
            files = list(filter(lambda x: 'raw' not in x.lower(), files))
            print(f"We got {len(files)} files.")

            # we've got the folder name from configuration previously
            print('dataset_directory_name', dataset_directory_name)

            # destination path
            destination_folder = os.path.join(data_save_folder_path, dataset_directory_name)
            print('destination_folder', destination_folder)

            # create destination folder
            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder) # create an empty folder
            else:
                # the folder already exists
                print("Destination folder exists. Re-creating it!")
                delete_non_empty_folder(destination_folder) # delete the folder
                os.makedirs(destination_folder) # create an empty folder

            # move files from tmp folder to target folder with os library
            for item in files:
                # save journals to subfolder within the specified target folder 
                os.rename(
                    os.path.join(root, item),
                    os.path.join(destination_folder, item))
            print("Files copied to final data folder")

            # remove the now unnecessary tmp folder
            delete_non_empty_folder(temporary_folder_path)
            print("DONE")

we got files without a folder in ../_data/data_Incontinence_NER_v5_20231208_tmp
We got 3000 files.
dataset_directory_name incontinence_journals
destination_folder ../_data/data_Incontinence_NER_v5_20231208/incontinence_journals
Files copied to final data folder
DONE


In [12]:
# load labels
dataset = Dataset.get_by_name(workspace, name=config[experiment]['labels_name'])
df = dataset.to_pandas_dataframe()
# This method is deprecated and will no longer be supported.
# Create a TabularDataset by calling the static methods on Dataset.Tabular and use the to_pandas_dataframe method there. For more information, see https://aka.ms/dataset-deprecation.

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [13]:
# save raw label data to file
df.to_csv(os.path.join(data_save_folder_path, 'raw_LABELINGS.csv'), sep='|', encoding='utf-16le', index=False)

In [14]:
# remove references that are no longer needed in order to manage memory
del df
del dataset
del workspace

### Process raw journals

#### Labels

In [15]:
labels_df = pd.read_csv(
    os.path.join(data_save_folder_path, 'raw_LABELINGS.csv'), 
    sep='|', 
    encoding='utf-16le')
labels_df.head(0)

,image_url,label,label_confidence,labeler,updated_by,labeling_time_in_seconds,label_creation_time


In [16]:
def get_short_filename(url:str) -> str:
    """extract filename from url"""
    filename = url.split('/')[-1]
    return filename

labels_df['short_filename'] = list(map(lambda x: get_short_filename(x), labels_df['image_url']))

In [17]:
# save the data frame to the local folder (overwrites the original file)
#labels_df.to_csv(os.path.join(data_save_folder_path, 'raw_LABELINGS.csv'), sep='|', encoding='utf-16le', index=False)

#### Data

In [18]:
def md5hash(input_string):
    result = hashlib.md5(input_string.encode())
    return result.hexdigest()


def extract_filename_from_filepath(url:str) -> str:
    # filename is the last of the string
    return url.split('/')[-1]


def get_start_index(text_id:str) -> str:
    return text_id.split('_')[-2]


def get_end_index(text_id:str) -> str:
    end_value = text_id.split('_')[-1]
    return end_value.split('.')[0]

In [19]:
print(data_save_folder_path, dataset_directory_name)

../_data/data_Incontinence_NER_v5_20231208 incontinence_journals


In [20]:
# get the journal files' paths
journal_filenames = []
for root, dirs, files in os.walk(os.path.join(data_save_folder_path, dataset_directory_name), topdown=False):
   for name in files:
      journal_filenames.append(os.path.join(root, name))

# filter out files that are not journals (only text files accepted)
journal_filenames = list(filter(lambda x: x.endswith('.txt'), journal_filenames))

# form tuples with path and file name
names_ids = [(journal_filename, extract_filename_from_filepath(journal_filename)) for journal_filename in journal_filenames]

In [21]:
# collect the nice data into this data frame
journals_df = pd.DataFrame(data=[], columns=['excerpt', 'filepath', 'text_id', 'start_index', 'end_index'])

# iterate the filenames and the content of the files 
objs = []
for name_id in names_ids:
    with open(name_id[0]) as f_in:
        text = f_in.readline()
        obj = {}
        obj['excerpt'] = text
        obj['filepath'] = name_id[0]
        file_name = str(name_id[1])
        obj['text_id'] = file_name.split('_')[0]
        obj['start_index'] = get_start_index(file_name)
        obj['end_index'] = get_end_index(file_name)
        obj['short_filename'] = name_id[1]
        objs.append(obj)

# add data to the data frame
journals_df = pd.concat([journals_df, pd.DataFrame.from_dict(objs)], ignore_index=True)

In [22]:
# process excerpts by removing pipe ('|') characters
journals_df['excerpt'] = list(map(lambda x: x.replace('|',';;;'), journals_df['excerpt']))

In [23]:
# # Write valid lines to compressed csv
# journals_df.to_csv(
#     os.path.join(data_save_folder_path,'tmp_TEXTS.csv'),
#     encoding='utf-8',
#     sep='|',
#     quoting=0,
#     header=True,
#     index=False
# )

In [24]:
print(f"journals: {journals_df.shape[0]}, labels: {labels_df.shape[0]}")

journals: 3000, labels: 2561


## Match the label records with journal records

In [25]:
# merge data frames 
dataset = pd.merge(left=labels_df, right=journals_df, how='inner', on='short_filename')
dataset.head(0) # display column headers

,image_url,label,label_confidence,labeler,updated_by,labeling_time_in_seconds,label_creation_time,short_filename,excerpt,filepath,text_id,start_index,end_index


In [26]:
dataset.info() # display statistics of separate columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2561 entries, 0 to 2560
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   image_url                 2561 non-null   object 
 1   label                     2561 non-null   object 
 2   label_confidence          2561 non-null   object 
 3   labeler                   2561 non-null   object 
 4   updated_by                349 non-null    object 
 5   labeling_time_in_seconds  2561 non-null   float64
 6   label_creation_time       2561 non-null   object 
 7   short_filename            2561 non-null   object 
 8   excerpt                   2561 non-null   object 
 9   filepath                  2561 non-null   object 
 10  text_id                   2561 non-null   object 
 11  start_index               2561 non-null   object 
 12  end_index                 2561 non-null   object 
dtypes: float64(1), object(12)
memory usage: 260.2+ KB


In [27]:
# # Write valid lines to compressed csv
# dataset.to_csv(
#     'tmp_RECORDS.csv',
#     encoding='utf-8',
#     sep='|',
#     quoting=0,
#     header=True,
#     index=False
# )

In [28]:
# remove unnecessary dataframes for memory management
del labels_df
del journals_df

## BIO labeling

### Transform semicolons back to pipes
Previously we changed pipe character ('|') to three semicolons characters (';;;'). Now we replace them again with the pipe character.

In [29]:
dataset['data'] = list(map(lambda x: x.replace(';;;', '|'), dataset['excerpt']))

### Add modified_labels and only_labels columns

In [30]:
def modified_labels(labels_array:str):
    """The modified labels are a list with single dictionary. 
    However, they seem to be strings in the dataset. Let's 
    transform label_array back to list and dictionary."""
    # There might be more than one label per each labels_array.
    # Those are separated by new line character ('\n'). We must 
    # replace '\n' with comma (',') to match list format.
    labels_array = labels_array.replace('\n', ',')
    # not the most secure thing to do but we expect not to 
    # encounter anything malicious in the labels
    return ast.literal_eval(labels_array)

# get modified labels
dataset['modified_labels'] = list(map(lambda x: modified_labels(x), dataset['label']))

In [31]:
dataset['only_labels'] = list(map(lambda x: [x[0]['label']], dataset['modified_labels']))

dataset['only_labels'] # display

0          [Ongelmia]
1       [Ei ongelmia]
2          [Ongelmia]
3          [Ongelmia]
4          [Ongelmia]
            ...      
2556    [Ei ongelmia]
2557       [Ongelmia]
2558       [Ongelmia]
2559       [Ongelmia]
2560       [Ongelmia]
Name: only_labels, Length: 2561, dtype: object

In [32]:
list(set(list(map(lambda x: x[0], dataset['only_labels']))))

['Ei ongelmia', 'Ongelmia']

## Make BIO labels

BIO labels are tags for each word/term in text that indicate whether the term begins the labeled sentence or is in the middle of the labeled sentence or ends the labeled sentence.
Here's an example of BIO labels:

bio_label_list = [
"O", 
"B-LONELINESS_LONELY",
"I-LONELINESS_LONELY",
"B-LONELINESS_NOTLONELY",
"I-LONELINESS_NOTLONELY",
] where 

'O' indicates a word or term that is not significant to the labeling of the piece of text. Most of the terms and words in text excerpts are marked as such. 'B' marks the beginning and 'I' marks any term intermitted in the piece of sentence that labeler has deemed to be enough to signify that the sentence matches the given label. We have not used the ending tag. In addition to the indicative starting character, the label is given (in cases 'B' and 'I') preseced by a dash character ('-'). Label itself can have underscore in its name as it has been used in this example. Please, do not deviate away from the labeling format "<Indicative character\>-<Dataset\>\_<specific label\>". 

The BIO labels get added to the text excerpts in this part of the code; however, instead of having them in the code I recommend that they are read from a configuration file specific to each dataset case (including different versions).

In [33]:
def get_BIO_label_list(experiment:str):
    """Get the BIO label list as a list from the configs.ini file for 
        the given experiment (experiment must match section name). Returns
        BIO_label_list or throws an error."""
    if "bio_label_list" in config[experiment]:
        # get config values as string
        s:str = config[experiment]["bio_label_list"]
        
        # check length of s
        if len(s) < 3:
            raise ValueError("bio_label_list in config file is too short.")
        else:pass

        # remove starting and closing brackets
        if s.startswith('['):
            s = s[1:]
        else:
            pass
        if s.endswith(']'):
            s = s[:-1]
        else: pass

        # we use the values of bio_label_list variable in section "BIO labeling" in this file
        bio_label_list = None
        if ',' in s:
            # split text and remove extra spaces from values
            s = s.split(',')
            bio_label_list = [x.replace(' ', '') for x in s]
        
        return bio_label_list
    else:
        raise NotImplementedError(f"Could not find {experiment} in config")


bio_label_list = get_BIO_label_list(experiment)
print(bio_label_list)

['O', 'B-Ongelmia', 'I-Ongelmia', 'B-Eiongelmia', 'I-Eiongelmia', '']


In [34]:
def split_text(text):
    # The regular expression matches word characters or punctuation marks
    return re.findall(r'\w+|[-.,!?;]', text)

In [35]:
punctuation_list = set(string.punctuation)
def bio_tagging(text, labeled_data):

    words = split_text(text)

    
    bio_tag_list = []
    dict_labeled_data = {}
    label_array = []
    for label_data in labeled_data:
        
        if label_data["label"] != 'O':
            
            #label = mapping_labels_to_english[label_data["label"]]
            label = label_data["label"]
            label_array.append(label)

            offsetStart = label_data["offsetStart"]
            offsetEnd = label_data["offsetEnd"]
            sample_text = text[offsetStart:offsetEnd]
            words_sample = split_text(sample_text)


            for i, word_sample in enumerate(words_sample):
                if word_sample not in punctuation_list: #if word len is < 3, omit
                    if i == 0:
                        dict_labeled_data[word_sample + "_" + label] = 'B-' + label   
                    else:
                        dict_labeled_data[word_sample + "_" + label] = 'I-' + label


    bio_tag = {}
    bio_tag_list = []
    
    for j, word in enumerate(words):
        mapped = False
        for label in label_array:

            if (word + "_" + label) in dict_labeled_data and dict_labeled_data[word + "_" + label]!="used":
                bio_tag_list.append(dict_labeled_data[word + "_" + label])
                bio_tag[word] = dict_labeled_data[word + "_" + label]
                dict_labeled_data[word + "_" + label] = "used"

                mapped = True
            
            if mapped:
                break

        if not mapped:
            bio_tag[word] = "O"
            bio_tag_list.append("O")


    return bio_tag_list, words


In [36]:
dataset[["bi_tags", "words"]] = dataset.apply(
    lambda row: pd.Series(bio_tagging(row['data'], row['modified_labels'])),
    axis=1
)
df = dataset.copy()
# Words is a list of the excerpt text separated by each word/punctuation. For example, ['Hello', ',', 'world', '!', 'How', 'are', 'you', '?']
# Bi_tags is a list of items found in bio_label_list matching the words list. For example, [O,O,O,O,O,O,O,O].
#print(df[["words", "bi_tags"]].head(10))

In [37]:
def test_bio_labeling(data, i):
    print(data["modified_labels"][i])
    print(data["words"][i])
    print(data["bi_tags"][i])

# a quick test to see whats in some record for the tags
#test_bio_labeling(df, 100)

In [38]:
df.columns.values.tolist()

['image_url',
 'label',
 'label_confidence',
 'labeler',
 'updated_by',
 'labeling_time_in_seconds',
 'label_creation_time',
 'short_filename',
 'excerpt',
 'filepath',
 'text_id',
 'start_index',
 'end_index',
 'data',
 'modified_labels',
 'only_labels',
 'bi_tags',
 'words']

In [39]:
## Saving the BIO labeled data to file(s)
#Rename file to match the context of the data!

# Write valid lines to parquet
df.to_parquet(
    os.path.join(data_save_folder_path, 'tmp_BIO_labeled_data.parquet'),
    index=True,
    use_dictionary=False
)